In [1]:
import util
import xarray 
import numpy as np
ds = util.get_dataset()
sst_ds = xarray.open_dataset("../data/sst.wkmean.1990-present.nc")
sst_ds = sst_ds.assign_coords(lon=(((sst_ds.lon + 180) % 360) - 180)) ## Use consistent longitude
train_storms,valid_storms,test_storms = util.train_validation_test(ds,seed=42)

In [1]:
import numpy as np 

def coriolis_parameter(lat):
    return 2*np.sin(np.deg2rad(lat))

def make_X_y(ds,selected_storms,timesteps=5):
    Xout = []
    yout =[]
    storms = []
    for storm in selected_storms:
        usa_pres = ds.usa_pres.loc[storm]
        usa_wind = ds.usa_wind.loc[storm]
        ## All enteries have 360 points.
        valid_coords = ~(np.isnan(usa_wind) | np.isnan(usa_pres))
        lat = ds.lat.loc[storm][valid_coords]
        lon = ds.lon.loc[storm][valid_coords]
        storm_speed = ds.storm_speed.loc[storm][valid_coords]
        storm_dir = ds.storm_dir.loc[storm][valid_coords]
        usa_pres = usa_pres[valid_coords]
        usa_wind = usa_wind[valid_coords]
        time = ds.time.loc[storm][valid_coords]
        cor_param = coriolis_parameter(lat)
        try:
            sst = sst_ds.sst.interp(time=time,lat=lat,lon=lon)
        except ValueError:
            continue
        if np.isnan(sst).any():
            continue

        X = np.transpose(np.array([usa_wind,usa_pres,storm_speed,storm_dir,cor_param,sst,lat,lon]))
        for i in range(0,len(usa_wind)):
            if i+timesteps+1>=len(usa_wind):
                break
            Xout.append(X[i:i+timesteps])
            yout.append(X[i+timesteps+1][:-4])
            storms.append(storm)
    return np.stack(Xout),np.stack(yout),np.array(storms)

In [2]:
TIMEPOINTS = 5
x_train,y_train,storm_train = make_X_y(ds,train_storms,TIMEPOINTS)
x_valid,y_valid,storm_valid = make_X_y(ds,valid_storms,TIMEPOINTS)

NameError: name 'ds' is not defined

### Training

In [ ]:
from xgboost import XGBRegressor
xgb_r = XGBRegressor(n_estimators=200,n_jobs=-1)
xgb_r.fit(x_train.reshape((x_train.shape[0],-1)),y_train)

